HOLO everyone. 
Today we are going to make an image classifier to determine its HOLO or not.
Let's get cereal!

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
model = Sequential()

# First coat
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 1), data_format='channels_first'))

# Second coat
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

# Taco 
model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [4]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train',  
        target_size=(150, 150),  
        batch_size=batch_size,
        class_mode='binary')  


validation_generator = test_datagen.flow_from_directory(
        'validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

# time to bake
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
# bake that shit for a few minutes


model.save('holo_model.h5')

Found 229 images belonging to 2 classes.
Found 76 images belonging to 2 classes.
Epoch 1/10
125/125 [==============================] - 57s 454ms/step - loss: 0.7304 - acc: 0.5712 - val_loss: 0.6388 - val_acc: 0.7237
Epoch 2/10
125/125 [==============================] - 59s 476ms/step - loss: 0.6255 - acc: 0.6657 - val_loss: 0.6497 - val_acc: 0.6842
Epoch 3/10
125/125 [==============================] - 56s 449ms/step - loss: 0.5295 - acc: 0.7468 - val_loss: 0.6674 - val_acc: 0.6711
Epoch 4/10
125/125 [==============================] - 63s 506ms/step - loss: 0.4493 - acc: 0.7915 - val_loss: 0.7916 - val_acc: 0.6579
Epoch 5/10
125/125 [==============================] - 59s 469ms/step - loss: 0.3763 - acc: 0.8392 - val_loss: 1.0022 - val_acc: 0.6711
Epoch 6/10
125/125 [==============================] - 61s 492ms/step - loss: 0.3403 - acc: 0.8606 - val_loss: 0.7295 - val_acc: 0.6711
Epoch 7/10
125/125 [==============================] - 66s 527ms/step - loss: 0.3023 - acc: 0.8785 - val_loss:

In [6]:
from IPython.display import display, HTML
from keras.models import load_model
from PIL import Image
import numpy as np

model = load_model('holo_model.h5')
result_dict = {val:key for (key, val) in train_generator.class_indices.items()}

for i in range(1, 7):
    img = load_img('test/beauty_guru_{}.jpg'.format(i))

    resized_img = img.resize( (150, 150), Image.BILINEAR )
    resized_img = np.reshape(resized_img,[1,150,150,3])
    # whaddaya think
    classes = model.predict_classes(resized_img)
    
    result_text = "It's {0}".format(' '.join(result_dict[classes[0][0]].split('_')).upper())
    display(HTML('''<h1>Is it real HOLO ?</h1><img style="width:50%" src='test/beauty_guru_{0}.jpg'><h1>{1}</h1>'''.format(i, result_text)))